# starmap gene expression cell typing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import anndata as an
bad_ch_pick = True
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
gene_set = sc.read_h5ad('./SourceData/gene_expression.h5ad')
ephys_set = sc.read_h5ad('./SourceData/electrophysiology.h5ad')
joint_set = sc.read_h5ad('./SourceData/gene_ephys_joint.h5ad')

In [ ]:
stages = gene_set.obs['stage'].unique()
marker_ = ['GATA4','MYH6','HCN4','ATP2A2','TNNT2','MYH7','VCL','RYR2']
sc.pl.umap(gene_set, color=marker_,use_raw=False,save='CorrelatedUMAP.pdf',cmap='Reds_r')

In [ ]:
sc.set_figure_params(vector_friendly=False)
sc.pl.umap(gene_set, color=['cell type'],palette = 'Set1',save='gene_cluster_umap.pdf',size=250)
sc.pl.umap(gene_set, color=['stage'], palette = 'rainbow',save='gene_stage_umap.pdf',size=250)


sc.pl.umap(ephys_set, color=['cell type'],palette = 'Set2',save='ephys_cluster_umap.pdf',size=250)
sc.pl.umap(ephys_set, color=['stage'], palette = 'rainbow',save='ephys_stage_umap.pdf',size=250)

sc.pl.umap(joint_set, color=['cluster'],palette = 'spring',save='joint_cluster_umap.pdf',size=250)
sc.pl.umap(joint_set, color=['stage'], palette = 'rainbow',save='joint_stage_umap.pdf',size=250)



In [ ]:
'''
This code was adapted from https://github.com/AllenInstitute/coupledAE-patchseq
Credit to original authors
'''
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import pylab
import matplotlib
from scipy.optimize import linear_sum_assignment
import plotly.graph_objects as go
import plotly.express as pex

def color_cm(cmap,NUM_COLORS,):
    color = []
    color_idx = 0
    cm = pylab.get_cmap(cmap)
    for i in range(NUM_COLORS):
        color.append(matplotlib.colors.to_hex(cm(1. * i / NUM_COLORS)))  # color will now be an RGBA tuple
    return color
def contingency(a, b, unique_a, unique_b):
    """Populate contingency matrix. Rows and columns are not normalized in any way.
    
    Args:
        a (np.array): labels
        b (np.array): labels
        unique_a (np.array): unique list of labels. Can have more entries than np.unique(a)
        unique_b (np.array): unique list of labels. Can have more entries than np.unique(b)

    Returns:
        C (np.array): contingency matrix.
    """
    assert a.shape == b.shape
    C = np.zeros((np.size(unique_a), np.size(unique_b)))
    for i, la in enumerate(unique_a):
        for j, lb in enumerate(unique_b):
            C[i, j] = np.sum(np.logical_and(a == la, b == lb))
    return C
def matrix_scatterplot(M, xticklabels, yticklabels,legend_name=None, xlabel='', ylabel='', fig_width=10, fig_height=14, scale_factor=500,save_dot=None):
    """Plots a matrix with points as in a scatterplot. Area of points proportional to each matrix element. 
    Suitable to show sparse matrices.

    Args:
        M (np.array): a 2D array
        xticklabels: label list
        yticklabels: label list
        fig_width (int): matplotlib figure width
        fig_height (int): matplotlib figure height
        scale_factor (float): scales the points by this value. 
    """
#     M = M / M.sum(axis=0)
    M_norm = M / M.sum(axis=0)
    Mplot = M_norm.copy()*scale_factor
#     Mplot = Mplot / Mplot.sum(axis=0)
    Mplot = np.flip(Mplot, axis=0)
    yticklabels.reverse()
    x = np.arange(0, M.shape[1], 1)
    y = np.arange(0, M.shape[0], 1)
    xx, yy = np.meshgrid(x, y)
    plt.figure(figsize=(fig_width, fig_height))
    cmap='Set1'
    color = color_cm(cmap,M.shape[0]+M.shape[1])
    colors = np.repeat(color[:M.shape[0]],M.shape[1])
    plt.scatter(np.ravel(xx), np.ravel(yy), s=np.ravel(Mplot), c=colors[::-1])
    

    ax = plt.gca()
    ax.set_xlim(np.min(x)-0.5, np.max(x)+0.5)
    ax.set_ylim(np.min(y)-0.5, np.max(y)+0.5)
    ax.set_xticks(x)
    ax.set_xticklabels(xticklabels, rotation=90)
    ax.set_yticks(y)
    ax.set_yticklabels(yticklabels, rotation=0)
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)

    ax.tick_params(color='None')
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)

    for tick in ax.get_yticklabels():
        #tick.set_fontname("DejaVu Sans Mono")
        tick.set_fontfamily('monospace')
        tick.set_fontsize(12)

    for tick in ax.get_xticklabels():
        tick.set_fontfamily('monospace')
        tick.set_fontsize(12)

    plt.grid(color='gray', linestyle='-', linewidth=0.5, alpha=0.4)
    plt.box(False)
    plt.tight_layout()
    l1 = plt.scatter([],[], s=scale_factor/4, edgecolors='none',c='gray')
    l2 = plt.scatter([],[], s=scale_factor*2/4, edgecolors='none',c='gray')
    l3 = plt.scatter([],[], s=scale_factor*3/4, edgecolors='none',c='gray')
    l4 = plt.scatter([],[], s=scale_factor, edgecolors='none',c='gray')

    labels = ["25%", "50%", "75%", "100%"]

    legend1 = plt.legend([l1, l2, l3, l4], labels, ncol=1, frameon=True, fontsize=30,
    handlelength=2, loc=[1.1, 0.1], borderpad = 1.8,
    handletextpad=1, title='matching percentage', scatterpoints = 1)
    ax = plt.gca().add_artist(legend1)
    l_co = []
    for cl in color[:M.shape[0]]:
        l_co.append(plt.scatter([],[], s=scale_factor/4, edgecolors='none',c=cl))

    labels = legend_name

    legend2 =plt.legend(l_co, labels, ncol=1, frameon=True, fontsize=18,
    handlelength=2, loc=[1.1, 0.8], borderpad = 1.8,
    handletextpad=1, title='matching percentage', scatterpoints = 1)
    

    if save_dot:
        plt.savefig(save_dot,bbox_extra_artists=(legend1,legend2), bbox_inches='tight')
    plt.show()
    return
def consistency_plots(t_type,e_type,t_name='T',e_name='E',re_order=True, scale_factor=500,xlabel=r'Transcriptomic type',
                       ylabel=r'Electrophysiology type',save_dot=None,save_river=None,save_cf = None,save_cf_n = None):
#     print(np.unique(e_type),np.unique(t_type))
    C_e_types = contingency(a=e_type,
                  b=t_type,
                  unique_a=np.unique(e_type),
                  unique_b=np.unique(t_type))
    #Assign labels of clusters based on 'best match' with transcriptomic celltype label
    row_ind,col_ind = linear_sum_assignment(-C_e_types)
    
    #print(np.unique(e_type),np.unique(t_type),C_e_types.shape)
    
    C_ordered = C_e_types[:,col_ind]
    order_y = np.unique(t_type)[col_ind]

    t_labels_matched = t_type.copy()
    e_labels_matched = e_type.copy()
    if re_order:
        for name in order_y:
            ind = t_type == name
            t_labels_matched[ind] = name

    C_TE_consistency = contingency(a=e_labels_matched,
                    b=t_labels_matched,
                    unique_a=np.unique(e_labels_matched),
                    unique_b=np.unique(t_labels_matched))

    ax_xticklabels = [t_name+'-{}{:>4s}'.format(x,'({:d})'.format(np.sum(t_labels_matched==x))) for x in np.unique(t_labels_matched)]
    ax_yticklabels = [e_name+'-{} ({:d})'.format(y,np.sum(e_labels_matched==y)) for y in np.unique(e_labels_matched)]
    legend_name= [e_name+'-{}'.format(x) for x in np.unique(e_labels_matched)]

    #print(np.unique(e_labels_matched),np.unique(t_labels_matched),C_TE_consistency.shape)
    matrix_scatterplot(M=C_TE_consistency,scale_factor=scale_factor,
                       xticklabels=ax_xticklabels,
                       yticklabels=ax_yticklabels,
                       xlabel=xlabel,
                       ylabel=ylabel,
                       legend_name= legend_name,
                       fig_width=8,fig_height=8,save_dot=save_dot)

    ax_xticklabels = [t_name+'-{}'.format(x) for x in np.unique(t_labels_matched)]
    ax_yticklabels = [e_name+'-{}'.format(y) for y in np.unique(e_labels_matched)]
    df_et = pd.DataFrame(C_TE_consistency, columns = ax_xticklabels,
                       index= ax_yticklabels)
    plt.figure()
    sn.heatmap(df_et)
#     plt.title('gene clustering VS ephys cluster ')
    if save_cf_n:
        plt.savefig(save_cf_n, bbox_inches='tight')
        
    plt.figure()
    sn.heatmap(df_et.div(df_et.sum(axis=1), axis=0))
#     plt.title('gene clustering VS  ephys cluster ')
    if save_cf:
        plt.savefig(save_cf, bbox_inches='tight')
   
    df_river = {'e type':[],'t type':[],'et value':[]}
    for t_cell in df_et.columns.to_list():
        for e_cell in df_et.index.to_list():
            df_river['e type'].append(e_cell)
            df_river['t type'].append(t_cell)
            df_river['et value'].append(df_et.loc[e_cell,t_cell])
    df_river = pd.DataFrame(df_river)

    all_nodes = df_river['e type'].unique().tolist() + df_river['t type'].unique().tolist()
    source_indices = [all_nodes.index(e_type) for e_type in df_river['e type']]
    target_indices = [all_nodes.index(t_type) for t_type in df_river['t type']]

    NUM_COLORS = len(all_nodes)
    cmap='Set1'
    colors = color_cm(cmap,NUM_COLORS)


    # colors = pex.colors.qualitative.D3

    # node_colors_mappings = dict([(node,np.random.choice(colors)) for node in all_nodes])
    node_colors_mappings = dict([(node,colors[idx]) for idx,node in enumerate(all_nodes)])
    node_colors = [node_colors_mappings[node] for node in all_nodes]
    edge_colors = [node_colors_mappings[node] for node in df_river['e type']]

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 20,
          thickness = 20,
          line = dict(color = "black", width = 1.0),
          label =  all_nodes,
          color =  node_colors,
        ),

        link = dict(
          source =  source_indices,
          target =  target_indices,
          value =  df_river['et value'],
          color = edge_colors,
    ))])

    fig.update_layout(title_text="e type to t type",
                      height=600,
                      font_size=10)
    if save_river:
        fig.write_image(save_river)
    fig.show()
    
    

In [ ]:
stage = gene_set.obs['stage'].to_numpy()
m_type = joint_set.obs['cluster'].to_numpy()
name1='M'
name2='stage'
re_order=False
consistency_plots(m_type,stage,t_name=name1,e_name=name2,re_order=True, scale_factor=2000,xlabel=r'Joint clustering type',
                       ylabel=r'Stage',save_dot='./figures/dot_joint_clustering.pdf',save_river='./figures/river_joint_clustering.pdf',
                  save_cf = './figures/joint_single_cf.pdf',save_cf_n = './figures/joint_single_cf_norm.pdf')



In [ ]:
stage = gene_set.obs['stage'].to_numpy()
t_type = gene_set.obs['cell type'].to_numpy()
name1='T'
name2='stage'

consistency_plots(t_type,stage,t_name=name1,e_name=name2,re_order=True, scale_factor=2000,xlabel=r'Transcriptomic type',
                       ylabel=r'Stage',save_dot='./figures/gene_single_dot.pdf',save_river='./figures/gene_single_river.pdf',save_cf = './figures/gene_single_cf.pdf',save_cf_n = 'gene_single_cf_norm.pdf')

In [ ]:
stage = ephys_set.obs['stage'].to_numpy()
t_type = ephys_set.obs['cell type'].to_numpy()
name1='E'
name2='stage'

consistency_plots(t_type,stage,t_name=name1,e_name=name2,re_order=True, scale_factor=2000,xlabel=r'Electrophysiology type',
                       ylabel=r'Stage',save_dot='./figures/ephys_single_dot.pdf',save_river='./figures/ephys_single_river.pdf',save_cf = './figures/ephys_single_cf.pdf',save_cf_n = 'ephys_single_cf_norm.pdf')

# pseudo time analysis

In [ ]:
colors=['b','g','y','r']
sns.displot(gene_set.obs,x='pseudo', hue='stage', kind='kde', fill=True,palette=colors ,rug=True)
plt.savefig('./figures/Gene_pseuro_Shadow.pdf')
plt.show()
sns.displot(ephys_set.obs,x='pseudo', hue='stage', kind='kde', fill=True,palette=colors ,rug=True)
plt.savefig('./figures/Ephys_pseuro_Shadow.pdf')
plt.show()
sns.displot(joint_set.obs,x='pseudo', hue='stage', kind='kde', fill=True,palette=colors ,rug=True)
plt.savefig('./figures/Joint_pseuro_Shadow.pdf')
plt.show()



In [ ]:
fvalues=[]
pvalues=[]
label=['gene_contact','ephys','joint']
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue  = stats.f_oneway(gene_set[gene_set.obs['stage']=='day12'].obs['pseudo'], 
                                gene_set[gene_set.obs['stage']=='day21'].obs['pseudo'],
                                gene_set[gene_set.obs['stage']=='day46'].obs['pseudo'],
                                gene_set[gene_set.obs['stage']=='day64'].obs['pseudo'])
fvalues.append(fvalue)
pvalues.append(pvalue)
fvalue, pvalue = stats.f_oneway(ephys_set[ephys_set.obs['stage']=='day12'].obs['pseudo'], 
                                ephys_set[ephys_set.obs['stage']=='day21'].obs['pseudo'],
                                ephys_set[ephys_set.obs['stage']=='day46'].obs['pseudo'],
                                ephys_set[ephys_set.obs['stage']=='day64'].obs['pseudo'])
fvalues.append(fvalue)
pvalues.append(pvalue)
fvalue, pvalue = stats.f_oneway(joint_set[joint_set.obs['stage']=='day12'].obs['pseudo'], 
                                joint_set[joint_set.obs['stage']=='day21'].obs['pseudo'],
                                joint_set[joint_set.obs['stage']=='day46'].obs['pseudo'],
                                joint_set[joint_set.obs['stage']=='day64'].obs['pseudo'])
fvalues.append(fvalue)
pvalues.append(pvalue)



label=['gene_contact','ephys','joint']
import scipy.stats as stats
stat_test_pseudo = pd.DataFrame(index=label)







In [ ]:
from sklearn.metrics import calinski_harabasz_score
metrc_score = []
metrc_score.append(calinski_harabasz_score(gene_set.X, gene_set.obs['cell type']))
metrc_score.append(calinski_harabasz_score(ephys_set.X, ephys_set.obs['cell type']))
metrc_score.append(calinski_harabasz_score(joint_set.X, joint_set.obs['cluster']))
stat_test_pseudo['calinski_harabasz_score']=metrc_score



In [ ]:
from sklearn.metrics import silhouette_score
from itertools import combinations
comb = list(combinations([0, 1, 2, 3], 2))
p_matrix_ttest=pd.DataFrame(data=np.ones([4,4]), index=stages, columns=stages)
# p_matrix_ttest[:] = np.nan
for co in comb:
    idx = gene_set.obs['stage'].isin([stages[co[0]],stages[co[1]]])
    p_matrix_ttest.iloc[[co[1]],[co[0]]] = silhouette_score(gene_set[idx].obsm['X_umap'],
                    gene_set[idx].obs['stage'].values)
    p_matrix_ttest.iloc[[co[0]],[co[1]]] = p_matrix_ttest.iloc[[co[1]],[co[0]]]

print(p_matrix_ttest)
fig,ax=plt.subplots()
plt.title('contact gene clustering score')
sns.heatmap(p_matrix_ttest,annot=True,ax=ax,cmap='YlGnBu', vmin=0, vmax=1, linewidths=0.7)
plt.savefig('./figures/silhouette contact gene clustering score.pdf')


comb = list(combinations([0, 1, 2, 3], 2))
p_matrix_ttest=pd.DataFrame(data=np.ones([4,4]), index=stages, columns=stages)
# p_matrix_ttest[:] = np.nan
for co in comb:
    idx = ephys_set.obs['stage'].isin([stages[co[0]],stages[co[1]]])
    p_matrix_ttest.iloc[[co[1]],[co[0]]] = silhouette_score(ephys_set[idx].X,
                    ephys_set[idx].obs['stage'].values)
    p_matrix_ttest.iloc[[co[0]],[co[1]]] = p_matrix_ttest.iloc[[co[1]],[co[0]]]

print(p_matrix_ttest)
fig,ax=plt.subplots()
plt.title('contact ephys clustering score')
sns.heatmap(p_matrix_ttest,annot=True,ax=ax,cmap='YlGnBu', vmin=0, vmax=1, linewidths=0.7)
plt.savefig('./figures/silhouette contact ephys clustering score.pdf')


comb = list(combinations([0, 1, 2, 3], 2))
p_matrix_ttest=pd.DataFrame(data=np.ones([4,4]), index=stages, columns=stages)
# p_matrix_ttest[:] = np.nan
for co in comb:
    idx = joint_set.obs['stage'].isin([stages[co[0]],stages[co[1]]])
    p_matrix_ttest.iloc[[co[1]],[co[0]]] = silhouette_score(joint_set[idx].X,
                    joint_set[idx].obs['stage'].values)
    p_matrix_ttest.iloc[[co[0]],[co[1]]] = p_matrix_ttest.iloc[[co[1]],[co[0]]]

print(p_matrix_ttest)
fig,ax=plt.subplots()
plt.title('joint clustering score')
sns.heatmap(p_matrix_ttest,annot=True,ax=ax,cmap='YlGnBu', vmin=0, vmax=1, linewidths=0.7)
plt.savefig('./figures/silhouette joint clustering score.pdf')


# neuron analysis

In [ ]:
overall_cells = sc.read_h5ad('./SourceData/CulturedNeuronGeneAll.h5ad')


In [ ]:
sc.set_figure_params(dpi_save=300,figsize=[4.0, 4.0])
sc.pl.umap(overall_cells, color=['cell type'],palette = 'rainbow', legend_loc='right margin',save='umap_neuron_all_type.png')
sc.pl.dotplot(overall_cells, ['SLC17A6','GAD1','GFAP','LUM'], groupby=f'cell type',swap_axes=True,
              save='All_neuron_non_neuron_dotplot_specific_marker_major.png')
marker_names=pd.DataFrame(overall_cells.uns['rank_genes_groups']['names']).head(5).to_numpy().ravel(order='F')
_,idx = list(np.unique(marker_names,return_index=True))
markers = marker_names[np.sort(idx)]
sc.pl.heatmap(overall_cells, markers, log=True,groupby='cell type',cmap='Reds'
              ,swap_axes=True,save='All_neuron_non_neuron_heat_map_major.pdf')



In [ ]:
neuron_expression = sc.read_h5ad('./SourceData/CulturedNeuronGenesRecord.h5ad')
neuron_ephys = sc.read_h5ad('./SourceData/CulturedNeuronEphysRecord.h5ad')
markers = ['SLC17A6','SATB2','NREP','GAD1','BHLHE22','PHOX2A']

sc.pl.heatmap(neuron_expression, markers, log=False,groupby=f'cell type',cmap='Reds'
              ,swap_axes=False,save='located_neuron_gene_heatmap_majortype.pdf')
sc.pl.heatmap(neuron_ephys, ['Fano_factor','firing_rate','recovery_slope','repolarization_slope','peak_trough_ratio','peak_to_valley'],
              log=False, swap_axes=False, groupby=f'cell type', cmap='bwr', 
              show=True,save='located_neuron_ephys_heatmap_majortype.pdf')

